In [15]:
import pandas as pd
import requests
from datetime import datetime
from bs4 import BeautifulSoup
import time
from concurrent.futures import ProcessPoolExecutor, as_completed

In [ ]:
df = pd.read_csv('searchList.csv')
sppList = df.values.tolist()
sppList

In [17]:
def sppSearch(searchName):
    t0 = time.time()
    url = "APIE KEY HERE"
    check = False
    for attempt in range(3):
        while check is False:
            try: 
                name = searchName[19].replace(" ","%20")
                name = '"'+name+'"'
                r = requests.get(url+name)
                soup = BeautifulSoup(r.content, 'lxml')
                result = soup.find_all('div',class_='gs_ab_mdw')[1].text
                check = len(result) > 1
            except Exception: 
                continue
            break
    t1 = time.time()
    dur = round(t1-t0,2)
    print(searchName[1],result,dur)
    return(searchName[1],result,dur)

In [ ]:
results = []

with ProcessPoolExecutor(max_workers=10) as executor:
    start = time.time()
    futures = {executor.submit(sppSearch, spp): spp for spp in sppList }
    for result in as_completed(futures):
        link = futures.get(result)
        try:
            data = result.result()
        except Exception as e:
            print(e)
        else:
            results.append(data)
    end = time.time()
    print("Time Taken: {:.6f}s".format(end-start))

In [ ]:
res = pd.DataFrame(results, columns = ['sciName','result','time'])
res

In [ ]:
res['count'] = res['result'].str.extract('(.*) result')
res['count'] = res['count'].str.replace(',','')
res['count'] = res['count'].str.replace('About ','')
res['count'] = res['count'].fillna(0)
res['count'] = pd.to_numeric(res['count'])

In [ ]:
res['count'] = res['result'].str.extract('(.*) result')
res

In [ ]:
res['count'] = res['count'].str.replace(',','')
res

In [ ]:
res['count'] = res['count'].str.replace('About ','')
res

In [ ]:
res['count'] = res['count'].str.replace(' ','')
res

In [ ]:
res['count'] = pd.to_numeric(res['count'])
res

In [ ]:
res[:754]

In [37]:
res.loc[753,['count']] = 2650

In [40]:
res.loc[1305,['count']] = 5320

In [42]:
res.loc[3509,['count']] = 2350

In [44]:
res.loc[6114,['count']] = 5850

In [46]:
res.loc[9006,['count']] = 1120

In [48]:
res.loc[9306,['count']] = 1780

In [50]:
res.loc[10208,['count']] = 21800

In [ ]:
res['count'] = res['count'].fillna(0)
res

In [52]:
res['count'] = pd.to_numeric(res['count'])

In [53]:
res.to_csv('ScholarResults20210322.csv')

In [ ]:
import seaborn as sns
sns.boxplot(data=res['time'])

In [ ]:
res['time'].describe()

In [ ]:
res

In [70]:
df = df.merge(res,how='left',left_on='Scientific Name',right_on='sciName')

In [73]:
df = df.drop(['searchName','sciName','result','time'],axis=1)

In [75]:
df = df.drop(['Unnamed: 0'],axis=1)

In [81]:
df.to_csv('AllNames20210322.csv',index=False)

In [84]:
an = df.groupby('Accepted Name',sort=False)['count'].sum().reset_index(name = 'Accepted Count')

In [ ]:
len(df['Accepted Name'].unique())

In [93]:
an = an.merge(df,how='left',left_on='Accepted Name',right_on='Scientific Name')

In [100]:
an = an.drop('count',axis=1)

In [103]:
an = an.drop(['Scientific Name','Accepted Name_y'],axis=1)

In [ ]:
an.columns.tolist()

In [106]:
an.columns = ['Scientific Name',
 'Literature Count',
 'Jepson Common Name',
 'Status',
 'Jepson Link',
 'CNPS Common Name',
 'Family',
 'Lifeform',
 'CRPR',
 'SRank',
 'CESA',
 'NatureServe Common Name',
 'Species Group',
 'NatureServe Global Rank',
 'NatureServe Rounded Global Rank',
 'U.S. Endangered Species Act Status',
 'Distribution',
 'View on NatureServe Explorer']

In [108]:
an.to_csv('AcceptedNames20210322.csv',index=False)